# Introduction

For Project 2, I decided to try out CNN on image classification. Having to watch an anime called the "Quinsential Quintuplet" which is a show of quintuplet having to be tutored by a guy as they were having bad grades. Apprently, they can be very hard to be tell apart if they all dress the same and only the granparent have a way to tell them apart. Hence, I will only be identifying two of the sisters out of the 5 as they are more popular towards the community and there are numerous of fan arts which means more data for me. ☆*:.｡.o(≧▽≦)o.｡.:*☆


# Feature Selection ٩(◕‿◕｡)۶	

Since I have tried playing with these datasets quite a while back, I realize that training the picture as whole is 100% not the ideal kind of situation as firstly, it will take up alot of time to train each image and secondly, if the image is not consistent, the model would not even know what the user wants to detect. Hence, for that purpose, i actually have crop out features that I want the model to "see" and recognize. 

For example, Miku, 1 of the 5 sister, is always constantly wearing a headphone. Hence, I would crop the photo that is inclusive of the headphone. This way, I think that when the model is training, it will know what it needs to detect as the "focus range" is more specified.

# Data Augmentation

Despite having the community to support the show and drawing those fan arts, the good ones that I found are very limited. Hence, to artifically create more dataset, we will have to augment most of the photos we have to increase the size of the dataset. Augmenting dataset means that the traning dataset are created by modifying versions of images in the dataset.

Despite the fact that there are many ways to augment an image such as playing with the image brightness, position, aspect ratio, shifts, flips, zooms, and much more. It must be clear that the choice of the specific data augmentation techniques used for a training dataset must be chosen carefully and within the context of the training dataset and knowledge of the problem domain so that it will acutally helps you and not make things worse.

Image data augmentation is typically only applied to the training dataset.

# Training of Model

# Importing Libraries

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


- matplotlib  to use the "Agg"  backend enabling us to save plots to disk — that’s your first nuance!
- openCV will be reading our image.

# PLEASE CHANGE TO NO ARGUMENTS NEEDED

$ python train_simple_nn.py --dataset animals --model output/simple_nn.model --label-bin output/simple_nn_lb.pickle --plot output/simple_nn_plot.png# PLEASE CHANGE TO NO ARGUMENTS NEEDED

In [3]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
    help="path to input dataset of images")
ap.add_argument("-m", "--model", required=True,
    help="path to output trained model")
ap.add_argument("-l", "--label-bin", required=True,
    help="path to output label binarizer")
ap.add_argument("-p", "--plot", required=True,
    help="path to output accuracy/loss plot")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -d DATASET -m MODEL -l LABEL_BIN -p PLOT
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset, -m/--model, -l/--label-bin, -p/--plot


SystemExit: 2

/home/oslost/.virtualenvs/conda_fake/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


- --dataset : The path to our dataset of images on disk.
- --model : Our model will be serialized and output to disk. This argument contains the path to the output model   file.
- --label-bin : Dataset labels are serialized to disk for easy recall in other scripts. This is the path to the output label binarizer file.
- --plot : The path to the output training plot image file. We’ll review this plot to check for over/underfitting of our data.

In [ ]:
Keras Tutorial: How to get started with Keras, Deep Learning, and PythonPython
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("dataset")))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, resize the image to be 32x32 pixels (ignoring
    # aspect ratio), flatten the image into 32x32x3=3072 pixel image
    # into a list, and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    # Add image to the data array
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# Splitting the data

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

Our class labels are currently represented as strings; however, Keras will assume that both:

Labels are encoded as integers
And furthermore, one-hot encoding is performed on these labels making each label represented as a vector rather than an integer
To accomplish this encoding, we can use the LabelBinarizer  class from scikit-learn:

In [ ]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

The input layer and first hidden layer are defined on Line 76. will have an input_shape  of 3072  as there are 32x32x3=3072  pixels in a flattened input image. The first hidden layer will have 1024  nodes.

The second hidden layer will have 512  nodes (Line 77).

Finally, the number of nodes in the final output layer (Line 78) will be the number of possible class labels — in this case, the output layer will have three nodes, one for each of our class labels (“cats”, “dogs”, and “panda”, respectively).

In [ ]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 75
 
# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [ ]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=EPOCHS, batch_size=32)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["plot"])

In [ ]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save(args["model"])
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

# Predict

$ python predict.py --image images/cat.jpg --model output/simple_nn.model \
    --label-bin output/simple_nn_lb.pickle --width 32 --height 32 --flatten 1

In [ ]:
# import the necessary packages
from keras.models import load_model
import argparse
import pickle
import cv2


 
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
    help="path to input image we are going to classify")
ap.add_argument("-m", "--model", required=True,
    help="path to trained Keras model")
ap.add_argument("-l", "--label-bin", required=True,
    help="path to label binarizer")
ap.add_argument("-w", "--width", type=int, default=28,
    help="target spatial dimension width")
ap.add_argument("-e", "--height", type=int, default=28,
    help="target spatial dimension height")
ap.add_argument("-f", "--flatten", type=int, default=-1,
    help="whether or not we should flatten the image")
args = vars(ap.parse_args())

You’ll need to explicitly import load_model  from keras.models  whenever you write a script to load a Keras model from disk. OpenCV will be used for annotation and display. The pickle  module will be used to load our label binarizer.

- --image : The path to our input image.
- --model : Our trained and serialized Keras model path.
- --label-bin : Path to the serialized label binarizer.
- --width : The width of the input shape for our CNN. Remember — you can’t just specify anything here. You need to specify the width that the model is designed for.
- --height : The height of the image input to the CNN. The height specified must also match the network’s input shape.
- --flatten : Whether or not we should flatten the image. By default, we won’t flatten the image. If you need to flatten the image, you should pass a 1  for this argument.

In [ ]:
# load the input image and resize it to the target spatial dimensions
image = cv2.imread(args["image"])
output = image.copy()
image = cv2.resize(image, (args["width"], args["height"]))
 
# scale the pixel values to [0, 1]
image = image.astype("float") / 255.0

In [ ]:
# check to see if we should flatten the image and add a batch
# dimension
if args["flatten"] > 0:
    image = image.flatten()
    image = image.reshape((1, image.shape[0]))

# otherwise, we must be working with a CNN -- don't flatten the
# image, simply add the batch dimension
else:
    image = image.reshape((1, image.shape[0], image.shape[1],
        image.shape[2]))

# check to see if we should flatten the image and add a batch
# dimension
if args["flatten"] > 0:
    image = image.flatten()
    image = image.reshape((1, image.shape[0]))
 
# otherwise, we must be working with a CNN -- don't flatten the
# image, simply add the batch dimension
else:
    image = image.reshape((1, image.shape[0], image.shape[1],
    image.shape[2]))

In [ ]:
# load the model and label binarizer
print("[INFO] loading network and label binarizer...")
model = load_model(args["model"])
lb = pickle.loads(open(args["label_bin"], "rb").read())
 
# make a prediction on the image
preds = model.predict(image)
 
# find the class label index with the largest corresponding
# probability
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]

In [ ]:
# draw the class label + probability on the output image
text = "{}: {:.2f}%".format(label, preds[0][i] * 100)
cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
# show the output image
cv2.imshow("Image", output)
cv2.waitKey(0)

<center><h1><b>References</b></h1><center>
    
    
1. Data Augmentation : https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

2. Keras with DL : https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/
